## Import Libraries 

In [3]:
pip install gender-guesser


Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import requests
import gender_guesser.detector as gender

## Load dataset

In [5]:
df = pd.read_feather("D:\combined_data.feather")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181477 entries, 0 to 181476
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   title       181477 non-null  object
 1   permalink   181477 non-null  object
 2   project     181477 non-null  object
 3   authors     181477 non-null  object
 4   recipients  181477 non-null  object
 5   date_from   181477 non-null  object
 6   date_to     181477 non-null  object
 7   api         181477 non-null  object
 8   og_text     180037 non-null  object
 9   text        181477 non-null  object
 10  period      181477 non-null  object
dtypes: object(11)
memory usage: 15.2+ MB


In [7]:
df.drop_duplicates(subset=['text'])

,title,permalink,project,authors,recipients,date_from,date_to,api,og_text,text,period
0,"Abigail Adams to Mercy Otis Warren, 4 March 1797",https://founders.archives.gov/documents/Adams/...,Adams Papers,"Adams, Abigail","Warren, Mercy Otis",1797-03-04,1797-03-04,https://founders.archives.gov/API/docdata/Adam...,\n \n my Dear Madam\n ...,my Dear Madam Quincy March 4th 1797 I received...,Adams Presidency
1,"John Adams to Abigail Adams, 5 March 1797",https://founders.archives.gov/documents/Adams/...,Adams Papers,"Adams, John","Adams, Abigail",1797-03-05,1797-03-05,https://founders.archives.gov/API/docdata/Adam...,"\n \n My dearest Friend,\n ...","My dearest Friend, Philadelphia March 5. 1797 ...",Adams Presidency
2,"John Quincy Adams to Louisa Catherine Johnson,...",https://founders.archives.gov/documents/Adams/...,Adams Papers,"Adams, John Quincy","Johnson, Louisa Catherine",1797-03-06,1797-03-06,https://founders.archives.gov/API/docdata/Adam...,\n \n The Hague March 6. 1797....,The Hague March 6. 1797. Since writing my last...,Adams Presidency
3,"Louisa Catherine Johnson to John Quincy Adams,...",https://founders.archives.gov/documents/Adams/...,Adams Papers,"Johnson, Louisa Catherine","Adams, John Quincy",1797-03-07,1797-03-07,https://founders.archives.gov/API/docdata/Adam...,\n \n London March 7th 1797\n ...,"London March 7th 1797 The package, containing ...",Adams Presidency
4,"Belinda Smith Clarkson to John Adams, 8 March ...",https://founders.archives.gov/documents/Adams/...,Adams Papers,"Clarkson, Belinda Smith","Adams, John",1797-03-08,1797-03-08,https://founders.archives.gov/API/docdata/Adam...,\n \n Dear Sir.\n New...,"Dear Sir. New York, March. 8th.: [1797] I hope...",Adams Presidency
...,...,...,...,...,...,...,...,...,...,...,...
181472,"Memorandum Books, 1793",https://founders.archives.gov/documents/Jeffer...,Jefferson Papers,"Jefferson, Thomas",,1793-01-01,1793-12-31,https://founders.archives.gov/API/docdata/Jeff...,\n 1793.\n \n \n ...,1793. Jan. 3. Lodged in bank of U. S. a quarte...,Washington Presidency
181473,"Memorandum Books, 1794",https://founders.archives.gov/documents/Jeffer...,Jefferson Papers,"Jefferson, Thomas",,1794-01-01,1794-12-31,https://founders.archives.gov/API/docdata/Jeff...,\n 1794.\n \n \n ...,1794. Jan. 1. Pd. James 8.D. pd. Carstairs for...,Washington Presidency
181474,"Memorandum Books, 1795",https://founders.archives.gov/documents/Jeffer...,Jefferson Papers,"Jefferson, Thomas",,1795-01-01,1795-12-31,https://founders.archives.gov/API/docdata/Jeff...,\n 1795.\n \n \n ...,1795. Jan. 1. Recd. from John Clarke of Bedfor...,Washington Presidency
181475,"Memorandum Books, 1796",https://founders.archives.gov/documents/Jeffer...,Jefferson Papers,"Jefferson, Thomas",,1796-01-01,1796-12-31,https://founders.archives.gov/API/docdata/Jeff...,\n 1796.\n \n \n ...,1796. Jan. 1. Gave my bond to Lucy Wood for £1...,Washington Presidency


In [8]:
#correcting some common inconsistencies.
# Replace ', & Fins' with ' & Fins'
df['authors'] = df['authors'].str.replace(', & Fins', ' & Fins')
df['recipients'] = df['recipients'].str.replace(', & Fins', ' & Fins')

# Replace ', & Cie.' with ' & Cie.'
df['authors'] = df['authors'].str.replace(', & Cie.', ' & Cie.')
df['recipients'] = df['recipients'].str.replace(', & Cie.', ' & Cie.')

# Replace ', & ' with ' | '
df['authors'] = df['authors'].str.replace(', & ', ' | ')
df['recipients'] = df['recipients'].str.replace(', & ', ' | ')

# Replace ', Jr.' with ' Jr.'
df['authors'] = df['authors'].str.replace(', Jr.', ' Jr.')
df['recipients'] = df['recipients'].str.replace(', Jr.', ' Jr.')

# Replace ', Sr.' with ' Sr.'
df['authors'] = df['authors'].str.replace(', Sr.', ' Sr.')
df['recipients'] = df['recipients'].str.replace(', Sr.', ' Sr.')

# Remove ' (business)'
df['authors'] = df['authors'].str.replace(' \(business\)', '', regex=True)
df['recipients'] = df['recipients'].str.replace(' \(business\)', '', regex=True)

C:\Users\User\AppData\Local\Temp\ipykernel_22696\244805881.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['authors'] = df['authors'].str.replace(', & Cie.', ' & Cie.')
C:\Users\User\AppData\Local\Temp\ipykernel_22696\244805881.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['recipients'] = df['recipients'].str.replace(', & Cie.', ' & Cie.')
C:\Users\User\AppData\Local\Temp\ipykernel_22696\244805881.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['authors'] = df['authors'].str.replace(', Jr.', ' Jr.')
C:\Users\User\AppData\Local\Temp\ipykernel_22696\244805881.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['recipients'] = df['recipients'].str.replace(', Jr.', ' Jr.')
C:\Users\User\AppData\Local\Temp\ipykernel_22696\244805881.py:19: FutureWarning: The default v

In [9]:
# Define a list of replacements to be made
replacements = [
    ('Hamilton, Alexander \(Lieutenant Colonel\)', 'Hamilton, Alexander'),
    ('Steuben, Major General', 'Steuben, Friedrich Wilhelm Ludolf Gerhard Augustin, Baron [von]'),
    ('Steuben, Friedrich Wilhelm Ludolf Gerhard Augustin, Baron von', 'Steuben, Friedrich Wilhelm Ludolf Gerhard Augustin, Baron [von]'),
    ('Steuben, Friedrich Wilhelm Ludolf Gerhard Augustin, baron von', 'Steuben, Friedrich Wilhelm Ludolf Gerhard Augustin, Baron [von]'),
    ('Steuben, Baron von', 'Steuben, Friedrich Wilhelm Ludolf Gerhard Augustin, Baron [von]'),
    ('Vergennes, Charles Gravier, comte de', 'Vergennes, Charles Gravier, Comte de'),
    ('Dumas, Charles-Guillaume-Frédéric', 'Dumas, Charles William Frederic'),
    ('Dumas, C. W. F.', 'Dumas, Charles William Frederic'),
    ('Boudinot, Elias Jr.', 'Boudinot, Elias'),
    ('Bowdinot, Elias', 'Boudinot, Elias'),
    ('Schuyler, Philip John', 'Schuyler, Philip'),
    ('Lee, Henry Jr.', 'Lee, Henry'),
    ('Nelson, Thomas Jr.', 'Nelson, Thomas'),
    ('Muhlenberg, John Peter Gabriel', 'Muhlenberg, Peter'),
    ('Harrison, Benjamin Sr.', 'Harrison, Benjamin'),
    ('Williams, Jonathan Jr.', 'Williams, Jonathan'),
    ('Wharton, Thomas Jr.', 'Wharton, Thomas'),
    ('Virginia Delegates in Congress', 'Virginia Delegates'),
    ('Virginia Delegates, American Continental Congress', 'Virginia Delegates'),
    ('Thaxter, John Jr.', 'Thaxter, John'),
    ('Schweighauser, John Daniel', 'Schweighauser, Jean-Daniel'),
    ('Rendon, Francisco', 'Rendón, Francisco'),
    ('Neufville, Jean de', 'Neufville, Jean [de]'),
    ('Neufville, Leendert de', 'Neufville, Leonard [de]'),
    ('McDougall, Maj. Gen. Alexander', 'McDougall, Alexander'),
    ('Malcom, Colonel William', 'Malcom, William'),
    ('Létombe, Philippe-André-Joseph de', 'Létombe, Philippe André Joseph de'),
    ('Johnson, Thomas Jr.', 'Johnson, Thomas'),
    ('Hamilton, Alexander (Lieutenant Colonel)', 'Hamilton, Alexander'),
    ('Greene, Nathaniel', 'Greene, Nathanael'),
    ('Gérard, Conrad-Alexandre', 'Gérard, Conrad Alexandre'),
    ('Genet, Edme-Jacques', 'Genet, Edmé Jacques'),
    ('Dana, Francis M.', 'Dana, Francis'),
    ('Cooke, Nicholas Sr.', 'Cooke, Nicholas'),
    ('Chaumont, Jacques Donatien, Leray de', 'Chaumont, Jacques-Donatien Le Ray de'),
    ('Church, William Singleton', 'Digges, Thomas'),
    ('Fitzpatrick, William', 'Digges, Thomas'),
    ('Dundas, T.', 'Digges, Thomas'),
    ('Ross, Timothy D.', 'Digges, Thomas'),
    ('Beaumarchais, Pierre-Augustin Caron de', 'Beaumarchais, Pierre Augustin Caron de'),
    ('Rochambeau, Comte de', 'Rochambeau, Jean-Baptiste Donatien de Vimeur, comte de'),
    ('Rochambeau, Jean-Baptiste-Donatien de Vimeur, comte de', 'Rochambeau, Jean-Baptiste Donatien de Vimeur, comte de'),
    ('Barbé de Marbois, François', 'Barbé-Marbois (Barbé de Marbois), François'),
    ('Barbé-Marbois, François de', 'Barbé-Marbois (Barbé de Marbois), François'),
    ('Barbé-Marbois, François', 'Barbé-Marbois (Barbé de Marbois), François'),
    ('Barbé-Marbois, Marquis de', 'Barbé-Marbois (Barbé de Marbois), François'),
    ('Barbé-Marbois, Pierre-François', 'Barbé-Marbois (Barbé de Marbois), François'),
    ('Marbois, François Barbé de', 'Barbé-Marbois (Barbé de Marbois), François'),
    ('Marbois, François Marquis de Barbé-Marbois', 'Barbé-Marbois (Barbé de Marbois), François'),
    ('La Vauguyon, Paul-François de Quélen de Stuer de Caussade, duc de', 'La Vauguyon, Paul François de Quélen de Stuer de Causade, Duc de'),
    ('Dubuysson des Aix, Charles-François, chevalier', 'Du Buysson des Aix, Charles-François, vicomte'),
    ('Destouches, —— (f. 1779–80)', 'Charles-René-Dominique Sochet'),
    ('Destouches, —— (f. 1780–1782)', 'Destouches, Charles-René-Dominique Sochet'),
    ('Destouches, ——', 'Destouches, Charles-René-Dominique Sochet'),
    ('Destouches, Chevalier', 'Destouches, Charles-René-Dominique Sochet'),
    ('Destouches, Charles-René-Dominique Sochet \(f. 1779–80\)', 'Destouches, Charles-René-Dominique Sochet'),
    ('Destouches, Charles-René-Dominique Sochet \(f. 1780–1782\)', 'Destouches, Charles-René-Dominique Sochet'),
    ('Chaumont, Jacques-Donatien Le Ray de', 'Chaumont, Jacques Donatien, Leray de'),
    ('Sartine, Antoine-Raymond-Gualbert-Gabriel de', 'Sartine, Antoine Raymond Jean Gualbert Gabriel de'),
    ('Armand, Charles', 'Armand (Armand-Charles Tuffin, marquis de La Rouërie)'),
    ('Armand Tuffin, Charles, marquis de La Rouërie', 'Armand (Armand-Charles Tuffin, marquis de La Rouërie)'),
    ('Estaing, Charles-Hector Theodat, comte d’', 'Estaing, Charles-Hector Théodat, comte d’'),
    ('Estaing, Charles-Hector, comte d’', 'Estaing, Charles-Hector Théodat, comte d’'),
    ('Estaing, Charles-Henri, comte d’', 'Estaing, Charles-Hector Théodat, comte d’'),
    ('Caracciolo, Domenico, Marchese di Villamaina', 'Caracciolo, Domenico, Marchesse di Villa Marina'),
    ('Arendt, Henry Leonard Philip, baron d’', 'Arendt, Henry Leonard Philip'),
    ('Trumbull, Jonathan Sr.', 'Trumbull, Jonathan'),
    ('Berubé de Costentin, ——', 'Costentin, Berubé de'),
    ('Rocquette, Jacques, Elsevier, T. A.', 'Rocqùette, J.,Th. A. Elsevier, & P. Th. Rocqùette,'),
    ('Dubbeldemuts, Adrianus', 'Dubbeldemuts, F. & A.'),
    ('Dubbeldemuts, Franco', 'Dubbeldemuts, F. & A.'),
    ('Parsons, Samuel Holden', 'Parsons, Samuel H.'),
    ('Stirling, Lord \(née William Alexander\)', 'Stirling, Lord (né William Alexander)'),
    ('Alexander, William Lord Stirling', 'Stirling, Lord (né William Alexander)'),
    ('Stirling, Major General', 'Stirling, Lord (né William Alexander)'),
    ('Alexander, William', 'Stirling, Lord (né William Alexander)'),
    ('La Lande & Fynje, de', 'La Lande & Fynje'),
    ('Horneca, Fizeaux & Cie.', 'Horneca, Fizeaux & Co.'),
    ('Staphorst, Nicholas & Jacob van', 'Staphorst, Nicolaas & Jacob van'),
    ('Ambler, Jaquelin \(Jacquelin\)', 'Ambler, Jacquelin'),
    ('La Luzerne, Anne César, Chevalier de', 'La Luzerne, Anne-César, chevalier de')
]

# Apply all replacements
for old, new in replacements:
    df[['authors', 'recipients']] = df[['authors', 'recipients']].replace(old, new, regex=True)

In [10]:
# Standardizing names in 'authors' and 'recipients' columns
replacements = {
    'Adams, Abigail (daughter of JA and AA)': 'Adams, Abigail Smith',
    'Smith, Abigail Adams': 'Adams, Abigail Smith',
    'Shaw, Elizabeth Smith | Peabody, Elizabeth Smith Shaw': 'Peabody, Elizabeth Smith Shaw',
    'Shaw, Elizabeth Smith': 'Peabody, Elizabeth Smith Shaw',
    'Smith, Elizabeth (1750-1815) | Peabody, Elizabeth Smith Shaw': 'Peabody, Elizabeth Smith Shaw',
    'Johnson, Louisa Catherine': 'Adams, Louisa Catherine Johnson',
    'Cranch, Mary Smith | Cranch, Richard': 'Cranch, Mary Smith',
    'Smith, Elizabeth (1750-1815) | Peabody, Elizabeth Smith Shaw': 'Peabody, Elizabeth Smith Shaw',
    'Amiel,Charlotte | Amiel, Peter': 'Amiel, Charlotte',
    'Banister, John Sr. | Banister, Anne Blair': 'Banister, Anne Blair',
    'Banister, John | Banister, Anne Blair': 'Banister, Anne Blair',
    'Bankhead, Charles Lewis | Bankhead, Ann Cary Randolph': 'Bankhead, Anne Cary Randolph',
    'Bankhead, Ann Cary Randolph': 'Bankhead, Anne Cary Randolph',
    'Barclay, Thomas | Barclay, Mary': 'Barclay, Mary',
    'Bingham, Ann': 'Bingham, Anne Willing',
    'Bland, Martha Daingerfield': 'Bland, Martha Dangerfield',
    'Bonneville, Margaret B.': 'Bonneville, Margaret',
    'Bourdic, Marie-Anne-Henriette Payan de L’Estang, baronne de': 'Bourdic, Marie-Anne-Henriette Payan de l’Estaing, baronne de',
    "Brillon de Jouy, Anne-Louise Boivin d’Hardancourt | Pagin, André-Noël": "Brillon de Jouy, Anne-Louise Boivin d'Hardancourt",
    "Brillon de Jouy, Anne-Louise Boivin d’Hardancourt | Franklin, Benjamin": "Brillon de Jouy, Anne-Louise Boivin d'Hardancourt",
    "Brillon de Jouy, Anne-Louise Boivin d’Hardancourt | Brillon de Jouy, Jacques": "Brillon de Jouy, Anne-Louise Boivin d'Hardancourt",
    'Bristow, Mary': 'Bristow, Mary Harding',
    'Brown, Elizabeth': 'Brown, Elizabeth Otis',
    'Bréhan, Marquise de | Moustier, Elénore François Elie, Comte de': 'Bréhan, Marquise de',
    'Ceracchi, Thérèse': 'Ceracchi, Therese',
    'Church, John | Church, Angelica': 'Church, Angelica Schuyler',
    'Church, Angelica': 'Church, Angelica Schuyler',
    'Condorcet, Marie-Jean-Antoine-Nicolas Caritat, marquis de': 'Condorcet, Marie Jean Antoine Nicolas de Caritat, Marquis de',
    'Copley, Susanna Clarke': 'Copley, Susanne Clarke',
    'Corny, Marguérite Victoire de Palerne de': 'Corny, Marguerite Victoire de Palerne de',
    'Cosway, Maria': 'Cosway, Maria Hadfield',
    'Cramer, Catherine de Wesselow': 'Cramer, Catherine Wesselowsky',
    'Cranch, Elizabeth': 'Cranch, Elizabeth Palmer',
    'Cranch, Elizabeth | Norton, Elizabeth Cranch': 'Cranch, Elizabeth Palmer',
    'Cranch, Lucy | Greenleaf, Lucy Cranch': 'Cranch, Lucy',
    'Custis, Eliza Parke': 'Custis, Elizabeth Parke',
    'Dana, Elizabeth': 'Dana, Elizabeth Ellery',
    'De Windt, John Peter | De Windt, Caroline Amelia Smith': 'De Windt, Caroline Amelia Smith',
    'Deblois, Ruth Hooper': 'Deblois, Ruth Hooper Dalton',
    'Deux-Ponts, Marianne Camasse, comtesse de Forbach': 'Deux-Ponts, Marianne Camasse',
    'Doradour, Comtesse de': 'Doradour, Madame de',
    'Douglas, Katherine Sproule': 'Douglas, Katherine Sprowle',
    'Drummond, Mrs.': 'Drummond, Agatha',
    'Dutartre, Madame —— Melin': 'Dutartre, Antoinette-Thérèse Melin',
    'Enville (Anville, Danville), Louise Elisabeth de La Rochefoucauld, Duchesse d’': 'Enville, Marie-Louise-Nicole-Elisabeth de La Rochefoucauld, duchesse d’',
    'Greene, Catharine': 'Greene, Catherine',
    'Hamilton, Elizabeth': 'Hamilton, Elizabeth Schuyler',
    'Hay, Katherine Farnham': 'Hay, Catherine Farnham',
    'Helvétius, Anne-Catherine de Ligniville d’Autricourt': 'Helvétius, Anne-Catherine de Ligniville',
    'Houdetot, Elisabeth-Françoise-Sophie de La Live de Bellegarde, comtesse d’': 'Houdetot, Elisabeth Françoise Sophie de La Live Bellegarde, Comtesse d’',
    'Huntingdon, Selina Hastings, countess of': 'Huntingdon, Selena Hastings, countess of',
    'Izard, Ralph | Izard, Alice': 'Izard, Alice Delancey',
    'Izard, Alice | Izard, Ralph': 'Izard, Alice Delancey',
    'Izard, Alice': 'Izard, Alice Delancey',
    'Johnson, Abigail Louisa Smith Adams': 'Adams, Louisa Catherine Johnson',
    'Lafreté, Jean-Jacques de | Lafreté, Angélique-Michèle-Rosalie Jogues de Martinville': 'Lafreté, Angélique-Michèle-Rosalie Jogues de Martinville',
    'Lafreté, Angélique-Michèle-Rosalie Jogues de Martinville | Lafreté, Angélique-Michèle-Rosalie Jogues de Martinville': 'Lafreté, Angélique-Michèle-Rosalie Jogues de Martinville',
    'Lewis, Mary': 'Lewis, Mary Walker',
    'Macaulay, Catharine': 'Macaulay, Catherine',
    'Madison, Dolley': 'Madison, Dolley Payne Todd',
    'Miller, Catharine': 'Miller, Catharine Littlefield Greene',
    'Montgomery, Dorcas': 'Montgomery, Dorcas Armitage',
    'Moore, Samuel Preston | Moore, Hannah': 'Moore, Hanna',
    'Morris, Ann Cary': 'Morris, Anne Cary Randolph',
    'Morris, Phebe P.': 'Morris, Phoebe P.',
    'Moustier, Elénore François Elie, Marquis de': 'Moustier, Elénore François Elie, Comte de',
    "Moustier, Eléanor-François-Elie, comte de'": 'Moustier, Elénore François Elie, Comte de',
    'Murray, Judith Sargent Stevens': 'Murray, Judith Sargent',
    'Oster, Madame ——': 'Oster, Madame',
    'Page, Margaret': 'Page, Margaret Lowther',
    'Page, Mary': 'Page, Mary Tayloe',
    'Paradise, Lucy': 'Paradise, Lucy Ludwell',
    'Peabody, Stephen | Peabody, Elizabeth Smith Shaw': 'Peabody, Elizabeth Smith Shaw',
    'Penn, Juliana': 'Penn, Lady Juliana Fermor',
    'Pini, Andrea Tozzi | Pini, Elisabetta Mazzei': 'Pini, Elisabetta Mazzei',
    'Powel, Samuel | Powel, Elizabeth Willing': 'Powel, Elizabeth Willing',
    'Powel, Elizabeth': 'Powel, Elizabeth Willing',
    'Randolph, Anne': 'Randolph, Anne Cary',
    'Randolph, Ann': 'Randolph, Anne Cary',
    'Rochambeau, Jeanne-Thérèse Tellès d’Acosta, comtesse de': 'Rochambeau, Jeanne-Thérèse de Telles d’Acosta, comtesse de',
    'Rose, Frances Taylor | Rose, Robert H.': 'Rose, Frances Taylor',
    'Rucker, Frances Randolph': 'Rucker, Frances Bland',
    'Rutledge, Harriet': 'Rutledge, Harriott',
    'Sargent, Winthrop | Sargent, Mary McIntosh': 'Sargent, Mary McIntosh',
    'Seam, Catherine Thomas | Seam, Henrietta Liston': 'Seam, Henrietta Liston',
    'Seymour, Hannah': 'Seymour, Hannah Rawle',
    'Sewall, Dorothy': 'Sewall, Dorothy Quincy Hancock',
    'Smith, Ann Eliza': 'Smith, Ann Eliza Bleecker',
    'Smith, Caroline': 'Smith, Caroline Amelia',
    'Smith, Caroline Amelia': 'De Windt, Caroline Amelia Smith',
    'Smith, Elizabeth (1750-1815) | Peabody, Elizabeth Smith Shaw': 'Peabody, Elizabeth Smith Shaw',
    'Smith, Elizabeth (1750-1815)': 'Peabody, Elizabeth Smith Shaw',
    'Smith, Julia': 'Smith, Julia Ball',
    'Smith, Louisa Catherine Adams': 'Adams, Louisa Catherine Johnson',
    'Staehelin, Anna | Staehelin, Johann August': 'Staehelin, Anna',
    'Steuben, Friedrich Wilhelm Ludolf Gerhard Augustin, Baron von': 'Steuben, Friedrich Wilhelm von',
    'Steuben, Freidrich Wilhelm Ludolf Gerhard Augustin, Baron von': 'Steuben, Friedrich Wilhelm von',
    'Steuben, Friedrich Wilhelm von | von Steuben, Friedrich Wilhelm Ludolf Gerhard Augustin': 'Steuben, Friedrich Wilhelm von',
    'Thomas, William | Thomas, Mary': 'Thomas, Mary',
    'Treadwell, Mary': 'Treadwell, Mary Cranch',
    'Trist, Elizabeth': 'Trist, Elizabeth House',
    'Trumbull, M.': 'Trumbull, Martha',
    'Trumbull, Mary | Trumbull, John': 'Trumbull, Mary',
    'Van Berckel, Pieter Johan | Van Berckel, Catharina Cornelia van Beresteyn': 'Van Berckel, Catharina Cornelia van Beresteyn',
    'Van der Kemp, Francina': 'Van der Kemp, Francina Margaretha de Haseth',
    'Van Hogendorp, Willem Cornelis Constantijn | Van Hogendorp, Maria Dibbets': 'Van Hogendorp, Maria Dibbets',
    'Van Horne, Mary': 'Van Horne, Mary Bowne',
    'Van Horn, Mary': 'Van Horne, Mary Bowne',
    'Varick, Elizabeth': 'Varick, Elizabeth Van Wyck',
    'Warren, Mercy Otis': 'Warren, Mercy',
    'Washington, Mary': 'Washington, Mary Ball',
    'Washington, Martha': 'Washington, Martha Custis',
    'Wayles, Elizabeth': 'Wayles, Elizabeth Eppes',
    'Williams, Frances Fowke | Williams, Otho H.': 'Williams, Frances Fowke',
    'Wingate, Mary': 'Wingate, Mary Smith',
    'Wright, Elizabeth': 'Wright, Elizabeth Darnall',
    'Wynkoop, John | Wynkoop, Judith Lefferts': 'Wynkoop, Judith Lefferts',
    'Yznaga, Mercedes de Santillane': 'Yznaga, Mercedes de Santillana',
    'Amiel, Charlotte | Amiel, Peter': 'Amiel, Charlotte',
    'Anterroches, MMe d’': 'Anterroches, Jeanne-Françoise Teyssier (Tessier) de Chaunay, comtesse d’',
    'Bache, Sarah Franklin': 'Bache, Sarah',
    "Brillon de Jouy, Anne-Louise Boivin d'Hardancourt": 'Brillon de Jouy, Anne-Louise Boivin d’Hardancourt',
    'Conway, Madame': 'Conway, Françoise-Antoinette-Jeanne Langlois du Bouchet, comtesse de',
    'Corny, Madame de': 'Corny, Marguerite Victoire de Palerne de',
    'Goddard, William | Goddard, Mary Katherine': 'Goddard, Mary Katherine',
    'Henderson, James L. | Henderson, Elizabeth': 'Henderson, Elizabeth',
    'Hewson, Mary Stevenson': 'Hewson, Mary',
    'Hewson, William | Hewson, Mary Stevenson': 'Hewson, Mary',
    'Humphreys, Ann Frances | Humphreys, David': 'Humphreys, Ann Frances Bulkeley',
    'Lloyd, Richard Bennett | Lloyd, Amelia Leigh': 'Lloyd, Amelia Leigh',
    'Lloyd, Richard Bennett | Lloyd, Mrs. Richard Bennett': 'Lloyd, Amelia Leigh', 
    'Rose, Frances Taylor Madison': 'Rose, Frances Taylor',
    'Sigourney, Lydia H.': 'Sigourney, Lydia Howard Huntley',
    'St. Wast, Madame —— Modenx de | St. Wast, Madame —— Modenx de': 'St. Wast, Madame —— Modenx de',
    'Strahan, William | Strahan, Margaret': 'Strahan, William | Strahan, Margaret Penelope',
    'Tessé, Adrienne Catherine de Noailles, Comtesse de':'Tessé, Adrienne-Catherine de Noailles, comtesse de',
    'Tessé, Madame de': 'Tessé, Adrienne-Catherine de Noailles, comtesse de',
    'Thompson, Elizabeth':'Thompson, Elizabeth Blair',
    'Van Winter, Lucretia Wilhelmina van Mercken': 'Van Winter, Lucretia Wilhelmina',
    'Van Winter, Nicholas Simon | Van Winter, Lucretia Wilhelmina': 'Van Winter, Lucretia Wilhelmina',
    'Welsh, Ann': 'Welsh, Anna',
    'Willis, Anna':'Willis, Ann Carter'
}

# Apply replacements
df['authors'] = df['authors'].replace(replacements)
df['recipients'] = df['recipients'].replace(replacements)

In [11]:

# define function to get gender
d = gender.Detector()

def get_gender(name):
    if name:  # Check if the name is not empty
        # Split by comma and strip any leading/trailing whitespace
        name_parts = [part.strip() for part in name.split(',')]
        if len(name_parts) > 1:
            first_name = name_parts[1].split()[0]  # Get the first word after the comma
        else:
            first_name = name_parts[0]

        gender = d.get_gender(first_name)
        # Handling cases where gender might be 'andy' (for androgynous)
        if gender == 'andy':
            gender = 'unknown'
        return gender
    else:
        return 'unknown'

In [12]:
# apply function to my data by using the column "authors"
df['gender'] = df['authors'].apply(get_gender)


In [13]:
df.groupby('gender')['authors'].count()

gender
female             4833
male             160969
mostly_female       438
mostly_male         203
unknown           15034
Name: authors, dtype: int64

In [14]:
mostly_f_gender = df[df['gender'] == 'mostly_female']

In [15]:
a_mostly_females = list(mostly_f_gender['authors'].unique())
a_mostly_females

['Cranch, Mary Smith',
 'Pryor, Mary',
 'Otis, Mary Smith Gray',
 'Huger, Mary E.',
 'Hall, Mary',
 'Lincoln, Mary Otis',
 'Williams, Mary',
 'Morris, Mary White',
 'Prior, Allyn',
 'Jefferson, Mary',
 'Eppes, Mary Jefferson',
 'Nicolson, Mary',
 'Lucas, Mary',
 'Fisher, Mary',
 'Hewson, Mary',
 'Parker, Mary',
 'Hopkinson, Mary',
 'Rich, Mary',
 'Goddard, Mary Katherine',
 'Bache, Mary',
 'Robinson, Beverley',
 'Thornton, Presley',
 'McCann, Mary',
 'Barclay, Mary',
 'Palmer, Mary',
 'Smith, Mary Rutledge',
 'Bryant, Mary',
 'Davies, Mary Ann',
 'Randolph, Beverley',
 'Bristow, Mary Harding',
 'Nevill, Presley',
 'Osmont, Le Chevalier d’',
 'Lewis, Mary Walker',
 'L’Ecossay, Le Mau de',
 'Bolling, Mary Jefferson',
 'Yard, Shandy',
 'Villandry, Le Breton de',
 'Randolph, Mary',
 'Negus, Mary',
 'Glenholmes, Mary',
 'Ingraham, Mary',
 'Heator, Mary',
 'Hazard, Mary and Void, Anna',
 'Hazard, Mary',
 'Osborne, Mary',
 'Payne, Mary Coles',
 'Band, Mary',
 'Barns, Lauren',
 'Wallden, Mary'

In [16]:
# List of authors to change gender from 'unknown' to 'female'
authors_to_change = [
    'Warren, Mercy', 'Tousard, Anne-Louis de', 'Lafayette, Marie-Joseph-Paul-Yves-Roch-Gilbert du Motier, marquis de', 
    'Felt, Abigal Adams Shaw', 'Van Berckel, Bacler (Madame)', 'La Marche, Madame,  de', 'Allison, Burgess', 
    'Dick, Lady Janet', 'Ralph, Miss ——', 'Martin, Miss', 'Morellet, abbé André', 'Biheron, Marie-Catherine', 
    'Dumas, Jean-Daniel', 'La Luzerne, Anne-César, chevalier de', 'Randolph, Peyton', 'Lewis, Fielding', 
    'Sorin, Aglaé-Marguerite Blavet du Marais', 'Darrell, Sampson', 'Bassett, Burwell', 'Riccobini, Marie-Jeanne de Heurles', 
    'Brillon de Jouy, Anne-Louise Boivin d’Hardancourt', 'Le Veillard, Geneviève-Elisabeth Belamy', 'Stockton, Annis Boudinot', 
    'Lafayette, Anastasie-Louise-Pauline du Motier de', 'Chavexult, —— (Mademoiselles)', 
    'Lafayette, Marie-Adrienne-Françoise de Noailles, marquise de', 'Doradour, Madame de', 'd’Houdetot, Madame', 
    'Gibelin, Abbé', 'Tessé, Adrienne-Catherine de Noailles, comtesse de', 'André, Abbé', 'Lausanne, Mademoiselle de', 
    'Guibert, Abbé', 'Moustier, Elénore François Elie, Comte de', 'Gaubert, Abbé', 'Oster, Madame', 'Colmare, Madame de', 
    'Bellanger, Madame Plumard de', 'De Tessé, Madame', 'Townsend, Madame', 'D’Anjouer, Madame', 'Duplessy, Madame', 
    'Brionne, Comtesse de', 'Le Roy, Madame', 'Langat, Mademoiselle Thomas de', 'Carysfort, Lady', 
    'Caradeux Lecaye, Madame de'
]

# Replace 'unknown' with 'female' for specific authors
df.loc[df['authors'].isin(authors_to_change), 'gender'] = 'female'


In [17]:
# List of authors to change gender from 'unknown' to 'male'
male_authors = [
    'Hobart, Thankful White Adams', 'Tufts, Cotton', 'Gerry, Elbridge', 'Watson, Elkanah', 'Coxe, Tench', 
    'Williamson, Collen', 'Scott, Gustavus', 'Lesz, Geffroy', 'Fitch, Eliphalet', 'Chaumont, LeRay de', 'Thompson, Capt. Thomas',
    'Pearson, Eliphalet', 'Walter, Rev. William', 'Ewing, Maskell', 'Osmun, Benajah', 'Bourne, R.', 'Wilson, Rev. James',
    'Ross, Mr.', 'Grasse, Alexandre-François-Auguste de Grasse-Rouville, comte de, marquis de Tilly', 'Webb, W.S.',
    'Pseudonym: “An Unfortunate Misled Man”', 'Cole, W. K.', 'Hopkins, Roswell', 'Bourne, Sylvanus', 'Briscoe, Hanson',
    'Sheredine, Upton', 'Phipps, Beriah', 'North, Mr.', 'Payson, Phillips', 'Russell, Eleazer', 'Legaux, C.', 'Senman, J.',
    'Navoni, Francois de', 'Wood, Abiel', 'Evans, Cadwalader', 'Sargent, Winthrop', 'Harris, Asale', 'Hamilton, Dr. Joseph',
    'Tufts, Turell', 'Mason, Stevens Thomson', 'Ogden, Uzal', 'Norton, Beriah', 'Blackberd, Mr.', 'Van Polanen, R.G.',
    'Tucker, St. George', 'Gibbon, J.', 'Lewis, J.', 'Peale, Rembrandt', 'Smith, Thorowgood', 'Catlett, Hanson',
    'Goodrich, Elizur', 'Stone, M. I.', 'Fleury, Latour Longnevoué, Marquise de', 'Adams, Boylston', 'Lawrence, Effingham',
    'Cook, Orchard', 'Colbert, Chevalier de', 'Hall, Josias Carvel', 'Irvine, Callender', 'Morris, Staats', 'Ames, Fisher',
    'Morris, Gouverneur', 'Fondey, Dowe J.', 'Woodruff, Hunloke', 'Howe, Bezaleel', 'North, Gershem', 'Brinley, Col. Thomas',
    'Peyton, Yelverton', 'Strong, Selah', 'Elton, Revell', 'Wadsworth, Decius', 'Morton, Washington', 'Vaublanc, Vincent-Marie-Viénot, comte de',
    'Ashmun, Phineas', 'Hitchcock, Peletiah', 'Archer, Starling', 'Pike, Zebulon M.', 'Foster, Abiel', 'Wright, Euel E.',
    'Tufts, Hall', 'Muse, Battaile', 'Read, Saunders A.', 'Lee, Ludwell', 'Burroughs, Newburgh', 'Mennis, Callohill',
    'Ball, Burgess Sr.', 'Washington, Bushrod', 'Rogers, Medad', 'La Colombe, Louis-Saint-Ange Morel, chevalier de',
    'Bassett, Burwell Jr.', 'Dumas, Gabriel-Mathieu (Guillaume-Mathieu), comte de', 'Mason, Thomson', 'Price, Uvedale',
    'Colston, Rawleigh', 'Buchan, eleventh earl of (David Stuart Erskine; 1742-1829)', 'Short, Peyton', 'Snyder, G. W.',
    'Leigh, Marmaduke', 'Ball, Burgess', 'Powell, Leven', 'Grasse, Alexandre-François-Auguste de Grasse-Rouville, comte de (marquis de Tilly)',
    'Thompson & Veitch', 'Thornton, Presly', 'Francis, Tench', 'Taylor, Hubbard', 'Snowden and McCorkle', 'Montesquieu',
    'Bordley, J. Beale', 'Strother, French', 'Trist, Hore Browse', 'Maury, Fontaine', 'Page, Mann', 'Sinclair, Sir John',
    'Madison, Bishop James', 'Skipwith, Fulwar', 'Louis of Parma', 'Palisot de Beauvois', 'Lord Somerville', 'Skipwith, Sir Peyton',
    'Beauvois, Palisot de', 'Freire, Cypriano Ribeiro, Chevalier', 'Tazewell, Littleton W.', 'Armistead, B. D.', 'Holladay, Waller',
    'Rumford, Count', 'Rodney, Caesar A.', 'Conrad & McMunn', 'Vaughan, Dr. John', 'Edwards, Pierpont', 'Peyton, Craven',
    'Bradford, Gamaliel', 'Pierce, Deacon Thomas', 'Colden, Cadwallader', 'Rowse, G.', 'Franklin, Abiah', 'Dalibard, Thomas-François',
    'Nollet, Jean-Antoine', 'Lotbinière, Michel-Alain Chartier de', 'Ashebriner, Harbanus', 'Halkett, Sir Peter', 'Stirling, Lord (né William Alexander)',
    'Beccaria, Giambatista', 'Whalley, Eyre', 'Brown, Henton', 'Cust, Sir John', 'Shervell, E.', 'Cooper, Grey', 'Hick, W.',
    'Pringle, Sir John', 'Coffin, Keziah Folger', 'Penn, Springett', 'Wilmot, Montagu', 'Byles, Mather', 'Sarsfield, Guy-Claude, comte de',
    'MacLeane, Lachlin', 'Le Roy, Jean-Baptiste', 'Bertier, Joseph-Etienne', 'Smith, Wright & Gray', 'Marshall, Humphry',
    'Bowen, Penuel', 'Sargent, Chambers and Co.', 'Bache, Theophylact', 'Rozier, abbé François', 'Saussure, Horace-Bénédict de',
    'Daubenton, Edmé-Louis', 'Elliott, Grey', 'Hubbart, Tuthill', 'Phipps, J. Warner', 'Howe, Lord', 'La Péronie (Peyroney, Peyrouny), William',
    'Eyre, Littleton', 'Corbin, Lettice', 'Norfleet, Marmaduke', 'Riddick, Mills', 'Carlyle & Adam', 'Washington, Lund',
    'Lewis, Fielding Jr.', 'Botetourt, Norborne Berkeley, Baron de', 'Madison, Rev. James', 'Tyler, Royall', 'Watson, Elkanah Jr.',
    'Brush, Eliphalet', 'Chavagnes, Bidé de', 'Mably, Abbé Gabriel Bonnot de', 'Carmarthen, the Marquis of', 'Reventlow, the Count Frederik von',
    'Pike, J. B.', 'Hodinpyl, P.', 'Webster, Pelatiah', 'Woronzow, Comte de', 'Pinto de Balsamão, Luiz', 'Calonne, Charles-Alexandre de',
    'Aragon, M. de', 'Dormer, Sir Clement Cotterell', 'Burrell, Sir Peter', 'Blagden, Sir Charles', 'Adams, Rev. Zabdiel',
    'Fleury, Donadieu De', 'Adams, Deacon Ebenezer', 'Haskell, Elnathan Jr.', 'Sargent, Epes', 'Burke, Aedanus', 'Mulligan, Hercules',
    'Kinloch, Cleland', 'Campbell, Mrs. J.', 'Bland, Theodorick', 'More, P. I.', 'Minnes, Callohill', 'Howell, Phineas',
    'Mercy-Argenteau, Florimond-Claude-Charles, comte de', 'Flandre de Brunville, François-Antoine de', 'Lenoir, Jean-Charles-Pierre',
    'Grand, Jean-François-Paul', 'Ducarne de Blangy, Jacques-Joseph', 'Genet, Edmond-Charles-Edouard', 'Villiers, Louis-Pierre Dufourny de',
    'Cadet de Vaux', 'Antoine-Alexis-François', 'Caffiéri, Jean-Jacques','Esmangart, Charles-François-Hyacinthe','Bourzeis, Jacques-Aimée de', 
    'Goëzmann de Thurn, Louis-Valentin','Saint-Olympe', 'Jean-Baptiste-César du Buc, chevalier de','Guffroy, Armand-Benoît-Joseph','Grand, Rodolphe-Ferdinand','Le Roy, Adrien-Jean-Baptiste',
    'Kéralio, Agathon Guynement, chevalier de', 'Séqueville, François-Pierre de','Bouttret Durivault fils','Leutre, Laurent-Jean-Antoine de', 'Delamardelle, Guillaume-Pierre-François',
    'Bonnefoy, Jean-Baptiste','Lalande, Joseph-Jérôme Le Français de Lalande','Vaudreuil, Louis-Philippe de Rigaud, marquis de', 'Breteuil, Louis-Auguste Le Tonnelier, baron de',
    'Gérard de Rayneval, Joseph-Mathias','Champion de Cicé, Jérôme-Marie', 'Dupaty, Charles-Marguerite-Jean-Baptiste Mercier','Du Ponceau, Jean-Michel',
    'Dubuysson, Charles-François, chevalier', 'Ostervald, Frédéric-Samuel', 'Brouttin Mollien de Sombres, Marie-Françoise-Dominique',
    'Cassini fils, Jacques-Dominique de','Marignac, Gabriel-Louis Galissard de','La Rochefoucauld, Louis-Alexandre, duc de La Roche-Guyon et de',
    'Despeintreaux, Jean-Baptiste-Charles Lucas','Luce de Lancival, Jean-Charles-Julien','Defay-Boutheroue, François-Simon',
    'Lafreté, Jean-Jacques', 'Dufriche de Valazé, Charles-Eléonor','Treuttel, Jean-Georges','L’Enfant, Pierre-Charles | Genton de Villefranche, Jean-Louis Ambroise de | Béchet de Rochefontaine, Etienne-Nicolas-Marie',
    'Toustain-Richebourg, Charles-Gaspard, vicomte de','Fleury, François-Louis Teissèdre de','Barbançon, Augustin-Jean-Louis-Antoine du Prat, comte de', 'Miollis, Sextius-Alexandre-François',
    'Murinais, Antonin-Victor-Augustin Auberjon, comte de', 'Hilliard d’Auberteuil, Michel-René', 'Lameth, Charles-Malo-François, comte de','Estaing, Charles-Hector Théodat, comte d’',
    'Dulau d’Allemans, Pierre-Marie, vicomte de', 'Rochambeau, Jean-Baptiste Donatien de Vimeur, comte de', 'Barras-Saint-Laurent, Jacques-Melchior, comte de','Vioménil, Antoine-Charles du Houx, baron de','Welch, Wakelin',
    'La Bretonnière, Louis-Bon-Jean de La Couldre, comte de','La Rouërie, Armand-Charles Tuffin, marquis de', 'Fock, Jean-Henric, baron de','Gambs, Jean-Daniel de',
    'Duportail, Antoine-Jean-Louis Le Bègue de Presle','Chastellux, François-Jean de Beauvoir, marquis de','Tilghman, Tench','Sumner, Jethro','Montmorency, Laval de',
    'Du Bouchet, Denis-Jean-Florimond Langlois de Mautheville, chevalier','Montesquieu, Charles-Louis de Secondat, baron de La Brède et de', 'Gimat, Jean-Joseph Soubadère de',
    'Washington, Thornton','Kersaint, Armand-Guy-Simon de Coëtnempren, comte de','Pusignan, Alexandre-César de Genevy de','Smith, Melancton','Fontenille, Fremyn, marquis de',
    'Hay, Udny', 'Montesquieu, Jean-Baptiste de','Jones, Ruthey','Symmes, W.','Châteaufort, chevalier d’Aristay','Holtzendorff, Louis-Casimir, baron de','Balt, Middleton', 'Otto, Louis-Guillaume (comte de Mosloy)','Grouchet, comte de',
    'Wuibert de Mézières, Antoine-Félix', 'Woodcock, Bancroft','Cheiza d’Artaignan, comte de', 'Oilliamson, Marie-Gabriel-Eléanor, comte d’', 'Mauduit du Plessis, Thomas-Antoine, chevalier de',
    'Bowler, Metcalf','Anterroches, Jeanne-Françoise Teyssier (Tessier) de Chaunay, comtesse d’','Bourdon de Vatry, Marc-Antoine', 'Stith, Buckner','Polk, Gilliss',
    'Essarts, comtesse des', 'Parson, Osborn','Brissot de Warville, Jacques-Pierre','Cottineau de Kerloguen, Denis-Nicolas', 'Wallace, Gustavus Brown','Clarkson, Levinus', 'Delany, Sharp', 'Lauzun, Armand-Louis de Gontaut Biron, duc de',
    'Laumoy, Jean-Baptiste-Joseph, comte de','Gouvion, Jean-Baptiste de','Fowler, Theodosius', 'Chase, J. Foy', 'Grasse-Tilly, François-Joseph-Paul, comte de',
    'Haskell, Elnathan', 'Stark, Bolling','Burnley, Hardin', 'Faure, Guillaume-Stanislaus','Wallace, Gustavus B.','Livingston, Muscoe', 'Joel, Beesly Edgar','Pio, Chevalier de', 'Garvey, Plowden W.',
    'Contée, B.', 'Boyer, Gazaigner de','Wuibert, Antoine-Félix', 'Sangrain, P. Tourtille', 'Gargaz, Pierre-André', 'Frichet, Curé','Morellet, Abbé', 'Acher, Jean-Baptiste',
    'Audibert, Dominique','La Perrière, Brac de','Chauvier, Père', 'Holmes, Abiel','Jones, Skelton', 'Hawkesbury, Lord', 'Jones, Meriwether', 'Pichon, Louis-André', 'Smith, Larkin','Silliman, Wyllys',
    'Clinton, DeWitt','Langdon, Woodbury','Lormerie, Louis-Philippe Gallot de','Ridgway, Coats', 'Bulkley, Chauncy','Dodge, Unite', 'Smith, O’Brien', 'Carr, Dabney', 'Tazewell, Littleton Waller',
    'Thompson, Smith'



    ]

# Replace 'unknown' with 'female' for specific authors
df.loc[df['authors'].isin(male_authors), 'gender'] = 'male'


In [18]:
# Replace 'mostly_female' with 'female' and 'mostly_male' with 'male'
df['gender'] = df['gender'].replace({'mostly_female': 'female', 'mostly_male': 'male'})

In [19]:
df.groupby('gender')['authors'].count()

gender
female       6259
male       165444
unknown      9774
Name: authors, dtype: int64

In [20]:
df.drop_duplicates(subset=['text'])

,title,permalink,project,authors,recipients,date_from,date_to,api,og_text,text,period,gender
0,"Abigail Adams to Mercy Otis Warren, 4 March 1797",https://founders.archives.gov/documents/Adams/...,Adams Papers,"Adams, Abigail","Warren, Mercy",1797-03-04,1797-03-04,https://founders.archives.gov/API/docdata/Adam...,\n \n my Dear Madam\n ...,my Dear Madam Quincy March 4th 1797 I received...,Adams Presidency,female
1,"John Adams to Abigail Adams, 5 March 1797",https://founders.archives.gov/documents/Adams/...,Adams Papers,"Adams, John","Adams, Abigail",1797-03-05,1797-03-05,https://founders.archives.gov/API/docdata/Adam...,"\n \n My dearest Friend,\n ...","My dearest Friend, Philadelphia March 5. 1797 ...",Adams Presidency,male
2,"John Quincy Adams to Louisa Catherine Johnson,...",https://founders.archives.gov/documents/Adams/...,Adams Papers,"Adams, John Quincy","Adams, Louisa Catherine Johnson",1797-03-06,1797-03-06,https://founders.archives.gov/API/docdata/Adam...,\n \n The Hague March 6. 1797....,The Hague March 6. 1797. Since writing my last...,Adams Presidency,male
3,"Louisa Catherine Johnson to John Quincy Adams,...",https://founders.archives.gov/documents/Adams/...,Adams Papers,"Adams, Louisa Catherine Johnson","Adams, John Quincy",1797-03-07,1797-03-07,https://founders.archives.gov/API/docdata/Adam...,\n \n London March 7th 1797\n ...,"London March 7th 1797 The package, containing ...",Adams Presidency,female
4,"Belinda Smith Clarkson to John Adams, 8 March ...",https://founders.archives.gov/documents/Adams/...,Adams Papers,"Clarkson, Belinda Smith","Adams, John",1797-03-08,1797-03-08,https://founders.archives.gov/API/docdata/Adam...,\n \n Dear Sir.\n New...,"Dear Sir. New York, March. 8th.: [1797] I hope...",Adams Presidency,female
...,...,...,...,...,...,...,...,...,...,...,...,...
181472,"Memorandum Books, 1793",https://founders.archives.gov/documents/Jeffer...,Jefferson Papers,"Jefferson, Thomas",,1793-01-01,1793-12-31,https://founders.archives.gov/API/docdata/Jeff...,\n 1793.\n \n \n ...,1793. Jan. 3. Lodged in bank of U. S. a quarte...,Washington Presidency,male
181473,"Memorandum Books, 1794",https://founders.archives.gov/documents/Jeffer...,Jefferson Papers,"Jefferson, Thomas",,1794-01-01,1794-12-31,https://founders.archives.gov/API/docdata/Jeff...,\n 1794.\n \n \n ...,1794. Jan. 1. Pd. James 8.D. pd. Carstairs for...,Washington Presidency,male
181474,"Memorandum Books, 1795",https://founders.archives.gov/documents/Jeffer...,Jefferson Papers,"Jefferson, Thomas",,1795-01-01,1795-12-31,https://founders.archives.gov/API/docdata/Jeff...,\n 1795.\n \n \n ...,1795. Jan. 1. Recd. from John Clarke of Bedfor...,Washington Presidency,male
181475,"Memorandum Books, 1796",https://founders.archives.gov/documents/Jeffer...,Jefferson Papers,"Jefferson, Thomas",,1796-01-01,1796-12-31,https://founders.archives.gov/API/docdata/Jeff...,\n 1796.\n \n \n ...,1796. Jan. 1. Gave my bond to Lucy Wood for £1...,Washington Presidency,male


In [21]:
# Create a dictionary for quick gender lookup based on the author column
gender_dict = df.set_index('authors')['gender'].to_dict()

# Function to assign gender to recipients
def assign_recipient_gender(recipient):
    return gender_dict.get(recipient, 'unknown')

# Create a new column 'recipient_gender' and apply the function
df['recipient_gender'] = df['recipients'].apply(assign_recipient_gender)

In [23]:
# Save to CSV
df.to_csv('founders_online.csv', index=False)

print("File saved successfully.")

File saved successfully.
